# Assemblage de la pipeline

In [3]:
import pandas as pd
from typing import no_type_check
import numpy as np 
import itertools
import matplotlib.pyplot as plt
import scipy as sc 
import csv 
import os 
from tqdm import tqdm

ModuleNotFoundError: No module named 'scipy'

### Mapping
Cette fonction permet de récupérer uniquement les données pertinentes et présente en quantité suffisante pour être analysées 

In [1]:
def get_pouls(df):
    """Fonction qui cree une série pouls à partir de df.FC préférentiellement, et sinon, df.Pouls
    Args:
        - df : pandas.DataFrame  -> df["FC"]
    Returns:
        - pandas.Series
    """
    serie_pouls  = df['FC']
    serie_pouls[serie_pouls.isna()] = df['Pouls'].loc[serie_pouls.isna()]

    return serie_pouls
    
def get_pression(df):
    """Fonction qui cree une série pression à partir de df['PAm'] préférentiellement, et sinon, df['PNIm'] et sinon, df['PBm']
    Args:
        - df : pandas.DataFrame  -> df['PAm']
    Returns:
        - pandas.Series
    """
    serie_pression = df['PAm']
    serie_pression[serie_pression.isna()] = df['PNIm'].loc[serie_pression.isna()]
    serie_pression[serie_pression.isna()] = df['PBm'].loc[serie_pression.isna()]
    
    return serie_pression


def get_temperature(df):
    """Fonction qui cree une série pression à partir de df['Temp'] préférentiellement, et sinon, df['Toeso'] et sinon, df['T1']
    Args:
        - df : pandas.DataFrame  -> df['Temp']
    Returns:
        - pandas.Series
    """
    serie_temperature = df['Temp']
    serie_temperature[serie_temperature.isna()] = df['Toeso'].loc[serie_temperature.isna()]
    serie_temperature[serie_temperature.isna()] = df['T1'].loc[serie_temperature.isna()]

    return serie_temperature

def mapping_pas4(df, bloc_4 = False):
    """Fonction qui cree une nouvelle dataframe bien mappée avec les colonnes Pouls Pression SpO2  FR, et hors bloc 4, Temperature
    La température au bloc 4 peut varier lors de refroidissements donc on ne la map pas
    Args:
        - df : pandas.DataFrame -> df["FC"],df['PAm'],df['Temp'], etc..
        - bool
    Returns:
        - pandas.DataFrame"""
    df.replace('AP',np.NaN, inplace = True)
    mapped_df = pd.DataFrame()
    mapped_df["seconde"] = df["seconde"]
    mapped_df["Pouls"] = get_pouls(df)
    mapped_df["Pression"] = get_pression(df)
    if not bloc_4:
        mapped_df["Temperature"] = get_temperature(df)
        mapped_df["SpO2"] = df["SpO2"]
        mapped_df["FR"] = df["FR"]
    else :
    #traitement du bloc 4
        cols = df.columns
        badliste = ["FC","Pouls","PAm", "PNIm", "PBm",'HEURE','DATE','LIT','NOM']
        cols = cols.drop(badliste)
        for col in cols:
            mapped_df[col] = df[col]
        
    mapped_df.dropna(axis = 1, how = 'all')
    
    return mapped_df.astype(float)

### Separation des patients

Nous avons constaté que les fichiers contenaient chacuns plusieurs patients, l'idée est donc d'écrire un programme qui sépare les différents patients

In [2]:
def column_test_patient(df):
    '''
    arg: df, DataFrame
    return: DataSerie of meaningful Series
    '''
    serie_patient = df['Pouls']
    serie_patient[serie_patient.isna()] = df['SpO2'].loc[serie_patient.isna()]
    serie_patient[serie_patient.isna()] = df['Pression'].loc[serie_patient.isna()]
    serie_patient[serie_patient.isna()] = df['FR'].loc[serie_patient.isna()]

    return serie_patient


def binar_list(df):
    '''
    arg: df, DataFrame
    return: list of presence of data (with 0,1 mask)
    '''
    patient_serie=column_test_patient(df)
    return patient_serie.notna().astype(int)


def list_index_patients(binar_list):
    '''
    arg: binar_list of presence, list
    return: list of tuple (key,length of sequence)
    '''
    return [
        (key, len(list(group))) for key, group in itertools.groupby(binar_list)
    ]


            
def noises_treatment(list_index_patient, threshold):
    '''
    arg: list of tuple (key,length of sequence), threshold (duration of bloc cleaning)
    return: list of tuple (key,length of sequence) without noises
    '''
    nouveau_index=[list_index_patient[0]]
    i=1
    while i <(len(list_index_patient)-1):
        if list_index_patient[i][1]<threshold:
            last_el = list(nouveau_index[-1])
            last_el[1]+=list_index_patient[i][1]
            last_el[1]+=list_index_patient[i+1][1]
            nouveau_index[-1]=tuple(last_el)
            i+=1
        else :
            nouveau_index.append(list_index_patient[i])
        i+=1
    if list_index_patient[-1][1]>=threshold :
        nouveau_index.append(list_index_patient[-1])
    else : 
        last_el = list(nouveau_index[-1])
        last_el[1]+=list_index_patient[-1][1]
        nouveau_index[-1]=tuple(last_el)
    return nouveau_index

def accumulate_list(list_len):
    return [0]+list(itertools.accumulate(list_len))

def break_finder(list_index_patients,threshold):
    '''
    arg: list of tuple (key,length of sequence) without noises
    return: list of list [begin,end] which limits index of cleaning
    '''
    list_len=[len1[1] for len1 in list_index_patients]
    accumulate_list1=accumulate_list(list_len)
    breaks=[]
    for index,len1 in enumerate(list_index_patients):
        value , length = len1
        begin , end = accumulate_list1[index] , accumulate_list1[index+1]
        if (value == 0) and (length > threshold):
            breaks.append([begin,end])
    return breaks



def list_to_patients(list_break,df):
    """d'une liste de listes debut/fin des pauses au bloc et de la dataframe associée renvoie la liste de dataframe patients 
    Args:
        - liste: list
        - df: pd.DataFrame
    Return
        patients: list de pd.Dataframe"""
    patients=[df[list_break[i][1]:list_break[i+1][0]] for i in range (-1,len(list_break)-1)] #On commence à -1 pour réserver une case pour le potentiel premier patient
    if list_break:
        patients[0]=df[0:list_break[0][0]]
        patients.append(df[list_break[-1][1]:-1])
    return patients


def separation_patients(df,threshold):
    '''
    arg: df, Dataframe ; threshold (duration of bloc cleaning)
    returns: list of patients' DataFrames
    '''
    list_index_patients1=list_index_patients(binar_list(df))
    list_index_pat_treated=noises_treatment(list_index_patients1, threshold)
    breaks=break_finder(list_index_pat_treated,threshold)

    return list_to_patients(breaks,df)

### Nettoyage

Nous avons besoin de quelques fonctions qui écartent les fichiers csv vides et les patients vides

In [4]:
def second(df):
    """renvoie une serie contenant le temps en seconde à partir de la colonne HEURE

    Args: 
        - df: pandas.DataFrame -> df contient les dates en heures 
    Return: 
        - pandas.Series """
    delta_serie = pd.to_datetime(df.HEURE) - pd.to_datetime(df.HEURE[0])
    delta_serie = delta_serie.apply(lambda delta : delta.total_seconds())
    
    while (delta_serie<0).any() :
        delta_serie.loc[delta_serie<0]+= 60*24*60
    
    return delta_serie

def df_is_empty(df):
    '''
    Args : 
        - df : Pandas.DataFrame
    Returns
        - bool 
    Renvoie True si la dataframe is empty, false sinon
    '''
    return len(df) < 5

def time_is_missing(df):
    '''
    Args : 
        - df : Pandas.DataFrame
    Returns
        - bool 
    Renvoie True si la dataframe ne contient pas de colonne 'HEURE'
    '''
    return 'HEURE' not in df.columns

def patient_is_empty(patient):
    '''
    Args : 
        - patient = Pandas.DataFrame
    Returns : 
        - bool 
    Retourne True si l'opération dure moins de 20 minutes
    '''
    return len(patient)<4*60

### De l'ordre 

Pour rendre le traitement des données plus agréable, nous avons trié les fichiers par blocs puis par dates 

In [ ]:
def sort_folder(folder):
    """prend le fichier data_brutes en arguments ( on celui avec les fichiers NaN enlevés ) 
    et renvoi un dictionnaire avec en clef le nom des blocs et en valeur une liste triée par date croissante 
    des csv des blocs
    Args :
        - folder : str ( nom du dossier)
    Return :
        - dic : dict """
    sorted_list = []
    dic = {}
    for filename in os.listdir(folder) :
        sorted_list.append((filename[0:7] + filename[-8:-4] + filename[-12:-10], filename))
    sorted_list.sort()
    
    for string, filename in sorted_list : 
        if filename[0:6] in dic.keys():
            
            dic[filename[0:6]].append(filename)
        else:
            dic[filename[0:6]]=[filename]
    return dic

### Pipeline

Il est maintenant temps de tout assembler pour écrire le programme qui va écrire les fichiers .csv patients 

Dans un premier temps, nous implémentons un dictionnaire qui associe à chaque bloc une durée minimale d'absence de données à partir de laquelle il faut considérer que le patient a été changé. 
Par exemple le bloc_1 ne traite qiue des petites chirurgies d'urgences, qui ne nécesite pas forcément un temps de transfert très long. Nous avons donc décidé d'abaisser le temps de battement à seulement 15 minutes. 

In [6]:
thresholds = {
        'bloc_1':15*60//5,
        'bloc_2':20*60//5,
        'bloc_3':20*60//5,
        'bloc_4':20*60//5,
        'bloc_5':20*60//5,
        'bloc_6':20*60//5,
        'bloc_7':20*60//5,
        'bloc_8':20*60//5,
        'Bloc_1':15*60//5,
        'Bloc_2':20*60//5,
        'Bloc_3':20*60//5,
        'Bloc_4':20*60//5,
        'Bloc_5':20*60//5,
        'Bloc_6':20*60//5,
        'Bloc_7':20*60//5,
        'Bloc_8':20*60//5
    }

In [9]:
def processing(folder):
    dic = sort_folder(folder)
    for bloc,filelist in dic.items():
        for filename in tqdm(filelist, desc = bloc):
            df = pd.read_csv(f"{folder}/{filename}", encoding = 'latin-1')
            if df_is_empty(df) or time_is_missing(df):
                continue
            df['seconde'] = second(df)
            df = mapping_pas4(df, bloc_4 = (bloc == "bloc_4"))
            for i,patient in enumerate(separation_patients(df,thresholds[bloc])):
                patient_name = filename.replace(".csv",f"_{i}.csv")
                patient_filename = "patients/" + patient_name
                if patient_is_empty(patient):
                    continue
                patient.to_csv(patient_filename)

On peut maintenant appliquer le pipeline à un dossier de fichiers csv. Cette pipeline va alors écrire des fichiers .csv des dans un dossier "patients" qu'il faut d'abord créer.

In [10]:
processing("Données")

NameError: name 'sort_folder' is not defined